# GPU for pytorch

In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

In [5]:
torch.cuda.get_device_name()

'Tesla T4'

In [6]:
torch.cuda.memory_allocated()

0

In [7]:
torch.cuda.memory_cached()

/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:416: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


0

In [8]:
a = torch.FloatTensor([1.0, 2.0])

In [9]:
a

tensor([1., 2.])

In [10]:
a.device

device(type='cpu')

## Use GPU and CUDA

In [12]:
a = torch.FloatTensor([1.0, 2.0]).cuda()

In [13]:
a.device

device(type='cuda', index=0)

In [14]:
torch.cuda.memory_allocated()

512

## Imports

In [18]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Creating Model

In [75]:
class Model(nn.Module):

  def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1)    # input leayer
    self.fc2 = nn.Linear(h1, h2)             # hidden layer
    self.out = nn.Linear(h2, out_features)   # output layer

  def forward(self, X):
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.out(X)
    return F.log_softmax(X, dim=1)

In [76]:
model = Model()

In [77]:
# our model is not on gpu
next(model.parameters()).is_cuda

False

### store on gpu

In [78]:
gpumodel = model.cuda()

In [79]:
next(gpumodel.parameters()).is_cuda

True

#### IRIS Dataset

In [80]:
from sklearn.datasets import load_iris

In [81]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns = iris.feature_names)

In [82]:
df['target'] = iris.target

In [83]:
df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [84]:
X = df.drop("target", axis=1).values

In [85]:
y = df["target"].values

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [87]:
# gpu and cuda
X_train = torch.FloatTensor(X_train).cuda()

In [88]:
X_test = torch.FloatTensor(X_test).cuda()

In [89]:
y_train = torch.FloatTensor(y_train).cuda()

In [90]:
y_test = torch.FloatTensor(y_test).cuda()

In [91]:
train_loader = DataLoader(X_train, batch_size=60, shuffle=True, pin_memory=True)
test_loader = DataLoader(X_test, batch_size=60, shuffle=False, pin_memory=True)

In [92]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

## Train Model

In [ ]:
import time
start_time = time.time()
epochs = 100
losses = []

for i in range(epochs):

  y_pred = gpumodel.forward(X_train)

  y_pred = y_pred.type(torch.LongTensor)   # casting to long
  y_train, y_pred = y_train.to(0), y_pred.to(0)

  loss = criterion(y_pred, y_train)
  losses.append(loss)

  if i%10 == 0:
    print(f"Epoch: {i} Loss: {loss.item()}")

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()



end = time.time() - start_time
print(f"TOTAL TIME: ", {end})


In [54]:
x = torch.Tensor([1,2])

In [55]:
x.dtype

torch.float32